# You have to design a Food Ordering app for a restaurant

In [ ]:
from flask import Flask, render_template, request, redirect, session
import sqlite3

app = Flask(__name__)
app.secret_key = 'your_secret_key'

# SQLite database connection
conn = sqlite3.connect('food_ordering.db')
cursor = conn.cursor()

# User registration
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        full_name = request.form['full_name']
        phone_number = request.form['phone_number']
        email = request.form['email']
        address = request.form['address']
        password = request.form['password']

        # Insert user data into the database (You should add validation and hashing of passwords)
        cursor.execute('''
            INSERT INTO Users (Full_Name, Phone_Number, Email, Address, Password)
            VALUES (?, ?, ?, ?, ?)
        ''', (full_name, phone_number, email, address, password))
        conn.commit()

        return redirect('/login')
    return render_template('register.html')

# User login
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']

        # Check user credentials (You should add password hashing)
        cursor.execute('SELECT Email, Password FROM Users WHERE Email = ? AND Password = ?', (email, password))
        user = cursor.fetchone()

        if user:
            session['user'] = email  # Store user's email in the session
            return redirect('/menu')
        else:
            return 'Invalid login credentials'

    return render_template('login.html')

# User menu - Place New Order
@app.route('/menu', methods=['GET', 'POST'])
def menu():
    if 'user' not in session:
        return redirect('/login')

    if request.method == 'POST':
        selected_items = request.form.getlist('food_item')
        # Process the selected items, possibly store them in the database as an order
        return 'Order placed successfully'

    # Fetch food items from the database
    cursor.execute('SELECT FoodID, Name, Quantity, Price FROM FoodItems')
    food_items = cursor.fetchall()

    return render_template('menu.html', food_items=food_items)

# User order history (You would need to create an Orders table in the database)
@app.route('/order-history')
def order_history():
    if 'user' not in session:
        return redirect('/login')

    # Fetch and display the user's order history
    return 'Order History Page'

# User profile update
@app.route('/update-profile', methods=['GET', 'POST'])
def update_profile():
    if 'user' not in session:
        return redirect('/login')

    if request.method == 'POST':
        # Update user's profile information in the database
        return 'Profile updated successfully'

    # Fetch user's profile data
    cursor.execute('SELECT Full_Name, Phone_Number, Email, Address FROM Users WHERE Email = ?', (session['user'],))
    user_data = cursor.fetchone()

    return render_template('update_profile.html', user_data=user_data)

if __name__ == '__main__':
    app.run(debug=True)